In [ ]:
import json
import time
import pprint
import boto3
import os
import re
from selenium import webdriver
from headless_chrome import create_driver
from botocore.exceptions import ClientError
from collections import defaultdict
from datetime import datetime, timezone

from selenium.webdriver import Chrome


from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


driver = Chrome()
driver.implicitly_wait(7)

brands = ["loro piana"]

In [ ]:
def scrape_articles():
    raw_articles = []
    baseUrl = "https://www.vinted.se/catalog?search_text={}&order=newest_first&catalog[]=5&page=1"

    for brand in brands:
        print(f"Scraping brand: {brand}")
        url = baseUrl.format(brand)
        driver.get(url)

        time.sleep(7)

        content = [
            element.get_attribute("outerHTML")
            for element in driver.find_elements(
                By.CLASS_NAME, "new-item-box__container"
            )
        ]

        print(len(content))
        raw_articles.append(content)

    print("----------------------")
    print(f"Scraped listings: {len(raw_articles)}")
    return raw_articles

In [93]:
articles = scrape_articles()

Scraping brand: loro piana
156
----------------------
Scraped listings: 1


In [97]:
articles[0][0]

'<div class="new-item-box__container" data-testid="product-item-id-6644420216"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6644420216--image" style="background-color: rgb(204, 207, 210);"><img alt="T-shirt elegancki stylowy polo Loro Piana M, varumärke: Loro Piana, skick: Ny med prislapp, storlek: M, 534.87 kr, 569.11 kr inkluderar köparskydd" class="web_ui__Image__content" data-testid="product-item-id-6644420216--image--img" src="https://images1.vinted.net/t/01_00eff_3n13GEFq8BZNbSRXcXD912bh/310x430/1751922202.jpeg?s=bc47a93aadc5abb5ccf35464aec24b6bf9ba53bd"></div></div><div class="u-position-absolute u-right u-bottom u-zindex-bump"><button aria-pressed="false" aria-label="Lägg till i favoriter" data-testid="product-item-id-664442021

In [ ]:
content = [
    element.get_attribute("outerHTML")
    for element in driver.find_elements(By.CLASS_NAME, "new-item-box__container")
]

In [ ]:
content[0].get_attribute("outerHTML")

'<div class="new-item-box__container" data-testid="product-item-id-6643727739"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6643727739--image" style="background-color: rgb(213, 212, 207);"><img alt="Wełniana marynarka Loro Piana, varumärke: Loro Piana, skick: Bra, storlek: L, 309.09 kr, 332.04 kr inkluderar köparskydd" class="web_ui__Image__content" data-testid="product-item-id-6643727739--image--img" src="https://images1.vinted.net/t/02_00e70_mGPR3MuTqBSFsPiCLbfot2oC/310x430/1751915314.jpeg?s=c88dc1d6822fddfe0bd716cf70c6a013b600f1af"></div></div><div class="u-position-absolute u-right u-bottom u-zindex-bump"><button aria-pressed="false" aria-label="Lägg till i favoriter, favoritmarkerad av 1 användare." data-testid="product-item-id-66

In [85]:
'<div class="new-item-box__container" data-testid="product-item-id-6643727739"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6643727739--image" style="background-color: rgb(213, 212, 207);"><img alt="Wełniana marynarka Loro Piana, varumärke: Loro Piana, skick: Bra, storlek: L, 309.09 kr, 332.04 kr inkluderar köparskydd" class="web_ui__Image__content" data-testid="product-item-id-6643727739--image--img" src="https://images1.vinted.net/t/02_00e70_mGPR3MuTqBSFsPiCLbfot2oC/310x430/1751915314.jpeg?s=c88dc1d6822fddfe0bd716cf70c6a013b600f1af"></div></div><div class="u-position-absolute u-right u-bottom u-zindex-bump"><button aria-pressed="false" aria-label="Lägg till i favoriter, favoritmarkerad av 1 användare." data-testid="product-item-id-6643727739--favourite" type="button" class="u-background-white u-flexbox u-align-items-center new-item-box__favourite-icon"><span class="web_ui__Icon__icon web_ui__Icon__greyscale-level-2" data-testid="favourite-icon" style="width: 16px;"><svg fill="none" viewBox="0 0 16 16" width="16" height="16" aria-hidden="true"><path fill="currentColor" d="M3.149 3.247c-1.03.662-1.462 1.67-1.392 2.79.073 1.146.68 2.425 1.797 3.477 1.608 1.515 3.4 2.968 4.31 3.688.081.064.19.064.271 0 .91-.72 2.702-2.173 4.31-3.688 1.117-1.052 1.725-2.331 1.798-3.476.07-1.12-.363-2.13-1.392-2.79-.576-.371-1.113-.498-1.591-.498-.673 0-1.317.366-1.843.819a6 6 0 0 0-.343.322l-.716.736a.5.5 0 0 1-.717 0l-.716-.736a5 5 0 0 0-.342-.322c-.526-.453-1.17-.819-1.843-.819-.48 0-1.015.127-1.591.497m-.811-1.262c.818-.526 1.636-.735 2.402-.735 1.2 0 2.186.634 2.822 1.182A7 7 0 0 1 8 2.845a7 7 0 0 1 .438-.413c.636-.548 1.621-1.182 2.822-1.182.765 0 1.583.21 2.402.735 1.529.983 2.18 2.535 2.078 4.147-.1 1.586-.92 3.206-2.267 4.474-1.654 1.559-3.485 3.043-4.407 3.772a1.715 1.715 0 0 1-2.132 0c-.922-.729-2.754-2.213-4.408-3.772C1.18 9.338.36 7.718.26 6.132.16 4.52.81 2.968 2.338 1.985"></path></svg></span><div class="web_ui__Spacer__small web_ui__Spacer__vertical"></div><span class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left">1</span></button><span aria-live="polite" class="u-visually-hidden">Borttagen.</span></div><a href="https://www.vinted.se/items/6643727739-welniana-marynarka-loro-piana?referrer=catalog" class="new-item-box__overlay new-item-box__overlay--clickable" data-testid="product-item-id-6643727739--overlay-link" title="Wełniana marynarka Loro Piana, varumärke: Loro Piana, skick: Bra, storlek: L, 309.09 kr, 332.04 kr inkluderar köparskydd"><div></div></a></div><div class="new-item-box__summary"><div class="web_ui__Cell__cell web_ui__Cell__tight" role="presentation"><div class="web_ui__Cell__content"><div class="web_ui__Cell__body"><div><div class="u-flexbox u-align-items-flex-start u-ui-padding-bottom-regular"><div class="u-min-width-none u-flex-grow"><div class="web_ui__Cell__cell web_ui__Cell__tight" role="presentation" data-testid="product-item-id-6643727739--description"><div class="web_ui__Cell__content"><div class="web_ui__Cell__body" data-testid="product-item-id-6643727739--description--content"><div class="u-flexbox u-justify-content-between"><div class="new-item-box__description"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__truncated" data-testid="product-item-id-6643727739--description-title">Loro Piana</p></div></div><div class="new-item-box__description"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__truncated" data-testid="product-item-id-6643727739--description-subtitle">L · Bra</p></div></div></div></div></div></div><div><div class="new-item-box__title" data-testid="product-item-id-6643727739--title-container"><div class="title-content"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__muted" data-testid="product-item-id-6643727739--price-text">309,09&nbsp;kr</p></div></div><div data-testid="product-item-id-6643727739--breakdown"><div class="u-flexbox u-align-items-flex-start"><button class="u-flexbox u-align-items-center u-flex-wrap" tabindex="0" aria-label="332,04&nbsp;kr inkluderar köparskydd" type="button"><span class="u-flexbox u-align-items-baseline u-flex-wrap"><span class="web_ui__Text__text web_ui__Text__subtitle web_ui__Text__left web_ui__Text__clickable web_ui__Text__underline-none">332,04&nbsp;kr</span><span class="web_ui__Spacer__x-small web_ui__Spacer__vertical"></span><span class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__clickable web_ui__Text__underline-none" tabindex="-1" data-testid="service-fee-included-title">inkl.</span></span><span class="web_ui__Spacer__x-small web_ui__Spacer__vertical"></span><span class="web_ui__Icon__icon web_ui__Icon__primary-default" data-testid="service-fee-included-icon" style="width: 12px;"><svg fill="none" viewBox="0 0 12 12" width="12" height="12" aria-hidden="true"><path fill="currentColor" d="m7.924 4.114.708.707-2.829 2.828-2.121-2.121.707-.707 1.414 1.414z"></path><path fill="currentColor" fill-rule="evenodd" d="M11 6c0 4.2-5 6-5 6s-5-1.8-5-6V1.8L6 0l5 1.8zM2 6V2.503l4-1.44 4 1.44V6c0 1.66-.98 2.902-2.115 3.787A9.4 9.4 0 0 1 6 10.917a9.368 9.368 0 0 1-1.885-1.13C2.981 8.902 2 7.66 2 6m3.66 5.06" clip-rule="evenodd"></path></svg></span></button></div><div class="web_ui__Spacer__small web_ui__Spacer__horizontal"></div></div></div></div></div></div></div></div></div>'

'<div class="new-item-box__container" data-testid="product-item-id-6643727739"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6643727739--image" style="background-color: rgb(213, 212, 207);"><img alt="Wełniana marynarka Loro Piana, varumärke: Loro Piana, skick: Bra, storlek: L, 309.09 kr, 332.04 kr inkluderar köparskydd" class="web_ui__Image__content" data-testid="product-item-id-6643727739--image--img" src="https://images1.vinted.net/t/02_00e70_mGPR3MuTqBSFsPiCLbfot2oC/310x430/1751915314.jpeg?s=c88dc1d6822fddfe0bd716cf70c6a013b600f1af"></div></div><div class="u-position-absolute u-right u-bottom u-zindex-bump"><button aria-pressed="false" aria-label="Lägg till i favoriter, favoritmarkerad av 1 användare." data-testid="product-item-id-66

In [ ]:
p = (
    articles[0]
    .find(
        "p",
        class_="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__muted",
        attrs={"data-testid": lambda x: x and "price-text" in x},
    )
    .text
)

p

'309,09\xa0kr'

In [42]:
p.contents

['£99.00']

In [23]:
articles[0]

<div class="feed-grid__item" data-testid="grid-item"><div class="feed-grid__item-content"><div class="u-flex-grow u-fill-width"><div class="new-item-box__container" data-testid="product-item-id-6642485319"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6642485319--image" style="background-color: rgb(215, 210, 206);"><img alt="Gio Genco Loro Piana Italy L mens cashmere coat, brand: Loro Piana, condition: Very good, size: L, £99.00, £104.65 includes Buyer Protection" class="web_ui__Image__content" data-testid="product-item-id-6642485319--image--img" src="https://images1.vinted.net/t/03_0244f_S4VNnoudNNf9gUTU2jFioJKZ/310x430/1751905811.jpeg?s=42c2bf9514b0f09a44ce34cc4e3fdd30d67549de"/></div></div><div class="u-position-absolute u-right u-bo

In [28]:
<div class="feed-grid__item" data-testid="grid-item"><div class="feed-grid__item-content"><div class="u-flex-grow u-fill-width"><div class="new-item-box__container" data-testid="product-item-id-6642485319"><div class="u-position-relative u-min-height-none u-flex-auto new-item-box__image-container"><div class="new-item-box__image"><div class="web_ui__Image__image web_ui__Image__cover web_ui__Image__portrait web_ui__Image__rounded web_ui__Image__scaled web_ui__Image__ratio" data-testid="product-item-id-6642485319--image" style="background-color: rgb(215, 210, 206);"><img alt="Gio Genco Loro Piana Italy L mens cashmere coat, brand: Loro Piana, condition: Very good, size: L, £99.00, £104.65 includes Buyer Protection" class="web_ui__Image__content" data-testid="product-item-id-6642485319--image--img" src="https://images1.vinted.net/t/03_0244f_S4VNnoudNNf9gUTU2jFioJKZ/310x430/1751905811.jpeg?s=42c2bf9514b0f09a44ce34cc4e3fdd30d67549de"/></div></div><div class="u-position-absolute u-right u-bottom u-zindex-bump"><button aria-label="Add to favourites, favourited by 6 users" aria-pressed="false" class="u-background-white u-flexbox u-align-items-center new-item-box__favourite-icon" data-testid="product-item-id-6642485319--favourite" type="button"><span class="web_ui__Icon__icon web_ui__Icon__greyscale-level-2" data-testid="favourite-icon" style="width: 16px;"><svg aria-hidden="true" fill="none" height="16" viewbox="0 0 16 16" width="16"><path d="M3.149 3.247c-1.03.662-1.462 1.67-1.392 2.79.073 1.146.68 2.425 1.797 3.477 1.608 1.515 3.4 2.968 4.31 3.688.081.064.19.064.271 0 .91-.72 2.702-2.173 4.31-3.688 1.117-1.052 1.725-2.331 1.798-3.476.07-1.12-.363-2.13-1.392-2.79-.576-.371-1.113-.498-1.591-.498-.673 0-1.317.366-1.843.819a6 6 0 0 0-.343.322l-.716.736a.5.5 0 0 1-.717 0l-.716-.736a5 5 0 0 0-.342-.322c-.526-.453-1.17-.819-1.843-.819-.48 0-1.015.127-1.591.497m-.811-1.262c.818-.526 1.636-.735 2.402-.735 1.2 0 2.186.634 2.822 1.182A7 7 0 0 1 8 2.845a7 7 0 0 1 .438-.413c.636-.548 1.621-1.182 2.822-1.182.765 0 1.583.21 2.402.735 1.529.983 2.18 2.535 2.078 4.147-.1 1.586-.92 3.206-2.267 4.474-1.654 1.559-3.485 3.043-4.407 3.772a1.715 1.715 0 0 1-2.132 0c-.922-.729-2.754-2.213-4.408-3.772C1.18 9.338.36 7.718.26 6.132.16 4.52.81 2.968 2.338 1.985" fill="currentColor"></path></svg></span><div class="web_ui__Spacer__small web_ui__Spacer__vertical"></div><span class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left">6</span></button><span aria-live="polite" class="u-visually-hidden">Removed!</span></div><a class="new-item-box__overlay new-item-box__overlay--clickable" data-testid="product-item-id-6642485319--overlay-link" href="https://www.vinted.co.uk/items/6642485319-gio-genco-loro-piana-italy-l-mens-cashmere-coat?referrer=catalog" title="Gio Genco Loro Piana Italy L mens cashmere coat, brand: Loro Piana, condition: Very good, size: L, £99.00, £104.65 includes Buyer Protection"><div></div></a></div><div class="new-item-box__summary"><div class="web_ui__Cell__cell web_ui__Cell__tight" role="presentation"><div class="web_ui__Cell__content"><div class="web_ui__Cell__body"><div><div class="u-flexbox u-align-items-flex-start u-ui-padding-bottom-regular"><div class="u-min-width-none u-flex-grow"><div class="web_ui__Cell__cell web_ui__Cell__tight" data-testid="product-item-id-6642485319--description" role="presentation"><div class="web_ui__Cell__content"><div class="web_ui__Cell__body" data-testid="product-item-id-6642485319--description--content"><div class="u-flexbox u-justify-content-between"><div class="new-item-box__description"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__truncated" data-testid="product-item-id-6642485319--description-title">Loro Piana</p></div></div><div class="new-item-box__description"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__truncated" data-testid="product-item-id-6642485319--description-subtitle">L · Very good</p></div></div></div></div></div></div><div><div class="new-item-box__title" data-testid="product-item-id-6642485319--title-container"><div class="title-content"><p class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__muted" data-testid="product-item-id-6642485319--price-text">£99.00</p></div></div><div data-testid="product-item-id-6642485319--breakdown"><div class="u-flexbox u-align-items-flex-start"><button aria-label="£104.65 includes Buyer Protection" class="u-flexbox u-align-items-center u-flex-wrap" tabindex="0" type="button"><span class="u-flexbox u-align-items-baseline u-flex-wrap"><span class="web_ui__Text__text web_ui__Text__subtitle web_ui__Text__left web_ui__Text__clickable web_ui__Text__underline-none">£104.65</span><span class="web_ui__Spacer__x-small web_ui__Spacer__vertical"></span><span class="web_ui__Text__text web_ui__Text__caption web_ui__Text__left web_ui__Text__clickable web_ui__Text__underline-none" data-testid="service-fee-included-title" tabindex="-1">incl.</span></span><span class="web_ui__Spacer__x-small web_ui__Spacer__vertical"></span><span class="web_ui__Icon__icon web_ui__Icon__primary-default" data-testid="service-fee-included-icon" style="width: 12px;"><svg aria-hidden="true" fill="none" height="12" viewbox="0 0 12 12" width="12"><path d="m7.924 4.114.708.707-2.829 2.828-2.121-2.121.707-.707 1.414 1.414z" fill="currentColor"></path><path clip-rule="evenodd" d="M11 6c0 4.2-5 6-5 6s-5-1.8-5-6V1.8L6 0l5 1.8zM2 6V2.503l4-1.44 4 1.44V6c0 1.66-.98 2.902-2.115 3.787A9.4 9.4 0 0 1 6 10.917a9.368 9.368 0 0 1-1.885-1.13C2.981 8.902 2 7.66 2 6m3.66 5.06" fill="currentColor" fill-rule="evenodd"></path></svg></span></button></div><div class="web_ui__Spacer__small web_ui__Spacer__horizontal"></div></div></div></div></div></div></div></div></div></div></div></div>

SyntaxError: invalid character '·' (U+00B7) (3712218741.py, line 1)

In [ ]:
x = articles[0]["currentColor"]

KeyError: 'currentColor'

In [2]:
! pip install vinted_scraper

In [3]:
from vinted_scraper import VintedScraper

scraper = VintedScraper("https://www.vinted.co.uk")  # init the scraper with the baseurl
params = {
    "search_text": "loro piana"
    # Add other query parameters like the pagination and so on
}
items = scraper.search(params)

8.0